In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dependencies = [
    "kagtool",
    "kaggle",
    "fastai",
    "nbdev",
    "datasets"
]

!pip install -U {" ".join(dependencies)}

from fastai.imports import *
from fastai.vision.all import *
import gc

In [ ]:
from kagtool.datasets.kaggle_downloader import KaggleDownloader

dataset_name = 'digit-recognizer'
creds = ''

path = KaggleDownloader(dataset_name, creds).load_or_fetch_kaggle_dataset()
df = pd.read_csv(path/'train.csv')
df.head()


In [ ]:
import pandas as pd
from fastai.vision.all import *
from PIL import Image

def get_image_from_pixels(row, img_size=(28, 28)):
    """
    Convert a row of pixel data to a PIL Image
    :param row: Pandas Series or array with pixel values.
    :param img_size: Tuple representing the size of the image.
    :return: PIL Image.
    """
    # Convert the row to a numpy array and reshape into image dimensions
    pixel_data = row.values.reshape(img_size)
    # Convert array to PIL Image
    return Image.fromarray(pixel_data.astype('uint8'), 'L') # 'L' mode for grayscale

In [ ]:
def get_x(row):
    if 'label' in row:
        return get_image_from_pixels(row[1:])  # Exclude the label column
    else:
        return get_image_from_pixels(row)

def get_y(row):
    return row['label']
    
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=get_x,  # Assuming get_x is defined as before
                   get_y=get_y,  # Assuming get_y is defined as before
                   splitter=RandomSplitter(valid_pct=0.2, seed=42),
                   item_tfms=Resize(128),  # Resize images
                   batch_tfms=aug_transforms(mult=2))  # Apply default augmentations, adjust as necessary
dls = dblock.dataloaders(df, bs=64)
dls.show_batch(max_n=9, figsize=(6, 7))


In [ ]:
arch = resnet34

In [ ]:
try:
    del learn
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

learn = vision_learner(dls, arch, metrics=accuracy).to_fp16()
learn.lr_find(suggest_funcs=(slide, valley))

In [ ]:
lr=0.003
epochs=10

In [ ]:
try:
    del learn
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

dls = dblock.dataloaders(df, bs=64, size=64)
learn = vision_learner(dls, arch, metrics=accuracy).to_fp16()

In [ ]:
learn.fine_tune(1, base_lr=lr)

In [ ]:
learn.show_results()

In [ ]:
# Plot confusion matrix to see where the model makes mistakes
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

# Plot top losses to see the images with the highest loss
interp.plot_top_losses(5, nrows=1)

In [ ]:
tst_df = pd.read_csv(path/'test.csv')
tst_dl = learn.dls.test_dl(tst_df)

In [ ]:
preds = learn.get_preds(dl=tst_dl)[0]

In [ ]:
first_batch = next(iter(tst_dl))
for i in range(3):
    first_batch[0][i].show()
    print("it should be a ", preds[i].argmax().item())

In [ ]:
dls = dblock.dataloaders(df, bs=64, size=64)
learn = vision_learner(dls, arch, metrics=accuracy).to_fp16()
learn.fine_tune(epochs, freeze_epochs=3, base_lr=lr)

In [ ]:
learn.recorder.plot_loss()

# Experimenting with Manual Transfer Learning & Discriminative Learning Rates

Check if we can get better results by manually applying unfreezing and discriminative learning rates

In [ ]:
learn.fine_tune??

In [ ]:
try:
    del learn
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

learn = vision_learner(dls, arch, metrics=accuracy).to_fp16()
# fastai already removed the head and froze the resnet layers for us
learn.fit_one_cycle(3, lr)

In [ ]:
learn.lr_find()
# we should ignore the valley
# we don't want the steepest anymore
# we are looking for a reasonable window before the spike

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(1e-4,5e-2))

In [ ]:
learn.recorder.plot_loss()

__fine_tune__ does the job just fine after all, let's just use that

In [ ]:
def ensemble():
    dls = dblock.dataloaders(df, bs=64, size=64)  # Start with smaller images
    learn = vision_learner(dls, arch, metrics=accuracy)
    # with learn.no_bar(),learn.no_logging(): learn.fine_tune(epochs, base_lr=lr)
    learn.fine_tune(12, freeze_epochs=3, base_lr=lr)
    return learn.get_preds(dl=tst_dl)[0]


learns = [ensemble() for _ in range(3)]
ens_preds = torch.stack(learns).mean(0)

In [ ]:
ens_preds.shape

In [ ]:
tst_df['ImageId'] = range(1, len(tst_df) + 1)
tst_df['Label'] = torch.argmax(ens_preds, dim=1)
sub_df = tst_df[['ImageId','Label']]
sub_df.to_csv('submission.csv', index=False)

# !head submission.csv

In [ ]:
# submit the file yourself! 
# the notebook doesn't automatically for you lol